# References
https://www.youtube.com/watch?v=ICXR9nDbudk

### Scrape Wikipedia

In [1]:
import pandas as pd
from geopandas.tools import geocode
from bs4 import BeautifulSoup
import requests
import urllib.request
import csv
import numpy as np
import folium 
import json
import googlemaps
import urllib
from datetime import datetime

In [4]:
wiki_url = 'https://en.wikipedia.org/wiki/Fire_Rescue_Victoria'

In [7]:
response = requests.get(wiki_url)

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
table = soup.findAll('table', attrs = {"class":"wikitable sortable"})

In [10]:
table

[<table class="wikitable sortable">
 <tbody><tr>
 <th rowspan="2">No
 </th>
 <th rowspan="2">Name
 </th>
 <th rowspan="2">District
 </th>
 <th colspan="3">FRV appliances
 </th>
 <th rowspan="2">Co-located with CFA
 </th></tr>
 <tr>
 <th>Pumper
 </th>
 <th>Aerial
 </th>
 <th>Specialist &amp; support
 </th></tr>
 <tr>
 <td>1
 </td>
 <td><a href="/wiki/Eastern_Hill_Fire_Station" title="Eastern Hill Fire Station">Eastern Hill</a>
 </td>
 <td>Central
 </td>
 <td>P1A, P1B, UP1
 </td>
 <td>LP1
 </td>
 <td>CU1, DC1A, DC1B, RHB1
 </td>
 <td>
 </td></tr>
 <tr>
 <td>2
 </td>
 <td>West Melbourne
 </td>
 <td>Central
 </td>
 <td>P2A, P2B
 </td>
 <td>
 </td>
 <td>
 </td>
 <td>
 </td></tr>
 <tr>
 <td>3
 </td>
 <td>Carlton
 </td>
 <td>Central
 </td>
 <td>P3
 </td>
 <td>
 </td>
 <td>R3
 </td>
 <td>
 </td></tr>
 <tr>
 <td>4
 </td>
 <td>Brunswick
 </td>
 <td>Central
 </td>
 <td>P4
 </td>
 <td>
 </td>
 <td>
 </td>
 <td>
 </td></tr>
 <tr>
 <td>5
 </td>
 <td>Broadmeadows
 </td>
 <td>Northern
 </td>
 <td>P5, 

In [11]:
df = pd.read_html(str(table))[0]

In [12]:
df.head()

No            Name  District FRV appliances                                \
  No            Name  District         Pumper Aerial   Specialist & support   
0  1    Eastern Hill   Central  P1A, P1B, UP1    LP1  CU1, DC1A, DC1B, RHB1   
1  2  West Melbourne   Central       P2A, P2B    NaN                    NaN   
2  3         Carlton   Central             P3    NaN                     R3   
3  4       Brunswick   Central             P4    NaN                    NaN   
4  5    Broadmeadows  Northern        P5, PT5    NaN                    NaN   

  Co-located with CFA  
  Co-located with CFA  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN

In [13]:
columns= []
for x in df.columns:
    columns.append(x[0])

In [14]:
df.columns = columns

In [15]:
df.head()

,No,Name,District,FRV appliances,FRV appliances,FRV appliances,Co-located with CFA
0,1,Eastern Hill,Central,"P1A, P1B, UP1",LP1,"CU1, DC1A, DC1B, RHB1",NaN
1,2,West Melbourne,Central,"P2A, P2B",NaN,NaN,NaN
2,3,Carlton,Central,P3,NaN,R3,NaN
3,4,Brunswick,Central,P4,NaN,NaN,NaN
4,5,Broadmeadows,Northern,"P5, PT5",NaN,NaN,NaN


In [16]:
df['Co-located with CFA']= df['Co-located with CFA'].replace(np.nan, "No")
df.head()

,No,Name,District,FRV appliances,FRV appliances,FRV appliances,Co-located with CFA
0,1,Eastern Hill,Central,"P1A, P1B, UP1",LP1,"CU1, DC1A, DC1B, RHB1",No
1,2,West Melbourne,Central,"P2A, P2B",NaN,NaN,No
2,3,Carlton,Central,P3,NaN,R3,No
3,4,Brunswick,Central,P4,NaN,NaN,No
4,5,Broadmeadows,Northern,"P5, PT5",NaN,NaN,No


In [17]:
df= df.replace(np.nan, "-")
df.head()

,No,Name,District,FRV appliances,FRV appliances,FRV appliances,Co-located with CFA
0,1,Eastern Hill,Central,"P1A, P1B, UP1",LP1,"CU1, DC1A, DC1B, RHB1",No
1,2,West Melbourne,Central,"P2A, P2B",-,-,No
2,3,Carlton,Central,P3,-,R3,No
3,4,Brunswick,Central,P4,-,-,No
4,5,Broadmeadows,Northern,"P5, PT5",-,-,No


In [18]:
df.columns = ['No.', 'Name','District', 'Pumper', 'Aerial', 'Specialist & Support', 'Co-located with CFA']

In [19]:
df.head()

,No.,Name,District,Pumper,Aerial,Specialist & Support,Co-located with CFA
0,1,Eastern Hill,Central,"P1A, P1B, UP1",LP1,"CU1, DC1A, DC1B, RHB1",No
1,2,West Melbourne,Central,"P2A, P2B",-,-,No
2,3,Carlton,Central,P3,-,R3,No
3,4,Brunswick,Central,P4,-,-,No
4,5,Broadmeadows,Northern,"P5, PT5",-,-,No


In [20]:
# Cleaning rows Burnley Complex and VEMTC

In [21]:
df.loc[7,['Specialist & Support', 'Pumper', 'Aerial', 'Specialist & Support']] = ['-','-','-','-']
df.loc[55,['Specialist & Support', 'Pumper', 'Aerial', 'Specialist & Support']] = ['-','-','-','-']

In [22]:
df.head()

,No.,Name,District,Pumper,Aerial,Specialist & Support,Co-located with CFA
0,1,Eastern Hill,Central,"P1A, P1B, UP1",LP1,"CU1, DC1A, DC1B, RHB1",No
1,2,West Melbourne,Central,"P2A, P2B",-,-,No
2,3,Carlton,Central,P3,-,R3,No
3,4,Brunswick,Central,P4,-,-,No
4,5,Broadmeadows,Northern,"P5, PT5",-,-,No


In [23]:
# save df
df.to_csv('wiki_table_frv.csv', index=False)

### Find Addresses using Google API

In [24]:
googlemaps_API_key ="-get-your-googleAPI-"

base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
AUTH_KEY = googlemaps_API_key

In [26]:
for i, row in df.iterrows():
    address = 'FRV fire brigade, ' + str(df.at[i,"Name"]) + ', Victoria, Australia'
    parameters = {"address" : address, 
             'key': AUTH_KEY}
    
    r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")
    data = json.loads(r.content)
    add = data['results'][0]['formatted_address']
    lat = data['results'][0]['geometry']['location']['lat']
    lng = data['results'][0]['geometry']['location']['lng']
    
    df.at[i,'add'] = add
    print(add)
    df.at[i,'lat'] = lat
    df.at[i,'lng'] = lng
    print(lat)
    print(lng)
    
df.to_csv('wiki_table_FRV_geo.csv')

1234 Yates St, Victoria, BC V8V 3M8, Canada
48.425096
-123.347728
60 Batman St, West Melbourne VIC 3003, Australia
-37.8106496
144.9510866
106 Bouverie St, Carlton VIC 3053, Australia
-37.8038817
144.9621337
24 Blyth St, Brunswick VIC 3056, Australia
-37.76594559999999
144.9650959
338 Camp Rd, Broadmeadows VIC 3047, Australia
-37.68626190000001
144.9292308
345A Gaffney St, Pascoe Vale VIC 3044, Australia
-37.73162130000001
144.9381796
92 Mahoneys Rd, Thomastown VIC 3074, Australia
-37.6933359
144.9994419
450 Burnley St, Richmond VIC 3121, Australia
-37.8293372
145.0057984
10 Somerton Park Dr, Campbellfield VIC 3061, Australia
-37.6444759
144.9463676
456 Albert St, East Melbourne VIC 3002, Australia
-37.8090171
144.9754069
28 Childs Rd, Epping VIC 3076, Australia
-37.660124
145.0257452
471 Bell St, Preston VIC 3072, Australia
-37.744617
144.9950081
24 Blyth St, Brunswick VIC 3056, Australia
-37.76594559999999
144.9650959
1083 Plenty Rd, Bundoora VIC 3083, Australia
-37.71085679999999
14

### Ploting the station on a Map

In [27]:
m = folium.Map(location = [-37.81421, 144.96316], tiles ='OpenStreetMap', zoom_start=7)

In [28]:
df = pd.read_csv('wiki_table_FRV_geo.csv')

In [29]:
for i, row in df.iterrows():
    lat = df.at[i, 'lat']
    lng = df.at[i, 'lng']
    
    popup = 'Name: ' + df.at[i, 'Name'] + '<br>' + 'District: '+ df.at[i, 'District'] + '<br>' + 'Pumper: ' + df.at[i, 'Pumper'] + '<br>' + 'Aerial: ' + df.at[i, 'Aerial'] + '<br>' + 'Specialist & Support: ' + df.at[i, 'Specialist & Support'] + '<br>' + 'Co-located with CFA: ' + df.at[i, 'Co-located with CFA']
    iframe = folium.IFrame(popup,
                       width=250,
                       height=150)

    popup = folium.Popup(iframe,
                     max_width=200)
    folium.Marker(location = [lat, lng], popup= popup, icon = folium.Icon(color='blue')).add_to(m)

m.save('wiki_FRV_Locations.html')